In [1]:
import json
from datetime import datetime
from openai import OpenAI
from playsound import playsound
from PIL import Image
from urllib.parse import urlparse
from urllib.request import urlopen
from IPython.display import Image
from IPython.core.display import HTML

from os.path import exists
import dotenv

dotenv.load_dotenv()

True

In [2]:
client = OpenAI()

In [3]:
task = "create a crm system for a small business"

In [54]:
project_manager = {
    "name": "Project Manager",
    "description": "You are a experienced project manager for custom software development.",
}

customer = {
    "name": "Customer",
    "description": "You are a owner of a small cafe. You have award a custom crm system project to the a company. You need to clarify you needs and requirements to the system analyst."
}

In [5]:
class Agent:
    def __init__(self, config: dict):
        self.config = config
        self.messages = []

    def send(self, message: str):
        self.messages.append({"role": "user", "content": [{"type": "text", "text": message}]})
        return self._send_to_llm()

    def add_init_message(self, message: str):
        self.messages.append({"role": "assistant", "content": message})

    def _get_system_message(self):
        return {"role": "system", "content": self.config.get("description", "You are a helpful AI assistant")}

    def _send_to_llm(self):
        messages = [self._get_system_message()] + self.messages
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        response_message = response.choices[0].message
        self.messages.append({"role": "assistant", "content": response_message.content})

        print(self.config.get('name', None), response_message.content + "\n\n")
        return response_message.content

In [57]:
pm_agent = Agent(project_manager)

In [58]:
team_structure = pm_agent.send(
    f"Your company just being award for the project: '{task}'. How the team structure will be?")

Project Manager The team structure for this project would be as follows:

1. Project Manager (You) - Responsible for overall project coordination, stakeholder communication, planning, resource allocation, and risk management.

2. Backend Developer 1 - Responsible for designing and developing the customer database with the specified fields, implementing the tracking of customer interactions, and developing the loyalty program management functionality.

3. Backend Developer 2 - Can be assigned to work alongside Backend Developer 1 to support in the development of the database and other functionalities as needed.

4. Quality Assurance (QA) Engineer - Responsible for testing the developed functionalities, ensuring functional and non-functional requirements are met, and identifying any defects or issues that need to be resolved.

5. User Interface (UI)/User Experience (UX) Designer - Responsible for designing and creating the user interface for the backend functionalities, ensuring a user-f

In [80]:
process = pm_agent.send(f"What is the process you will follow to complete the project?")

Project Manager To complete the project, I would follow a well-structured project management process. Here's a high-level overview of the key stages and activities involved:

1. Initiation:
   - Define project objectives, scope, and success criteria with stakeholders.
   - Identify project team members and their respective roles and responsibilities.
   - Conduct a kickoff meeting to align the team and stakeholders on project goals, timelines, and deliverables.

2. Planning:
   - Create a detailed project plan outlining tasks, dependencies, resources, and timelines.
   - Define the technical requirements for the customer database, customer interaction tracking, and loyalty program management.
   - Identify and mitigate potential risks.
   - Establish communication and reporting processes.

3. Design:
   - Work with the backend developer to create the database schema and design the structure for customer interactions and loyalty program management.
   - Collaborate with the UI/UX design

In [81]:
process

"To complete the project, I would follow a well-structured project management process. Here's a high-level overview of the key stages and activities involved:\n\n1. Initiation:\n   - Define project objectives, scope, and success criteria with stakeholders.\n   - Identify project team members and their respective roles and responsibilities.\n   - Conduct a kickoff meeting to align the team and stakeholders on project goals, timelines, and deliverables.\n\n2. Planning:\n   - Create a detailed project plan outlining tasks, dependencies, resources, and timelines.\n   - Define the technical requirements for the customer database, customer interaction tracking, and loyalty program management.\n   - Identify and mitigate potential risks.\n   - Establish communication and reporting processes.\n\n3. Design:\n   - Work with the backend developer to create the database schema and design the structure for customer interactions and loyalty program management.\n   - Collaborate with the UI/UX design

In [72]:
def json_parser(message: str):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        tool_choice={"type": "function", "function": {"name": "team_parser"}},
        tools=[{
            "type": "function",
            "function": {
                "name": "team_parser",
                "description": "Parse the team structure",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "members": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "name": {
                                        "type": "string",
                                        "description": "The name of the role of the team member, e.g. 'Project Manager'"
                                    },
                                    "description": {
                                        "type": "string",
                                        "description": "The responsibility of the team member, e.g. 'You are a experienced project manager for custom software development. Your are Responsible for overall project coordination, stakeholder communication, planning, resource allocation, and risk management.'"
                                    }
                                }
                            }
                        }
                    }
                },
            }
        }],
        messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message

In [73]:
team_structure_parsed = json_parser(team_structure)

In [77]:
team_json = json.loads(team_structure_parsed.tool_calls[0].function.arguments)

In [78]:
agents = [Agent(c) for c in team_json["members"]]

In [79]:
agents

In [32]:
sa_agent.add_init_message("Hello, I am a system analyst. I am here to help you design a system for your business.")

counter = 0
message = "Hello, I am a system analyst. I am here to help you design a system for your business."

message_log = []

while counter < 10:
    message = customer_agent.send(message)
    message_log.append({"role": "sa", "content": message})
    message = sa_agent.send(message)
    message_log.append({"role": "customer", "content": message})
    counter += 1

Customer Great, thank you for your assistance. I own a small cafe and I'm looking to implement a custom CRM system to better manage our customer relationships and improve our overall operations. I have a few key requirements that I'd like to discuss with you.

Firstly, I'd like the CRM system to have a customer database where we can store and manage customer information such as contact details, preferences, purchase history, and any special notes or interactions we've had with them.

I also want the system to have the capability to track customer interactions, such as emails, phone calls, and in-person visits, so that we can easily refer back to previous conversations and have a complete view of our interactions with each customer.

Additionally, I'd like the CRM to have a feature for managing loyalty programs and tracking customer rewards and incentives. This should include the ability to set up and manage different loyalty tiers and promotions, as well as the ability to track points 

In [33]:
sa2 = {
    "name": "System Analyst",
    "description": "You are a system analyst. Base on the conversation between a business analyst and customer, you need to create software requirements specification document for your development team.",
}

In [34]:
sa_agent_2 = Agent(sa2)

In [35]:
history = "\n".join([f"{m['role']}: {m['content']}" for m in message_log])
conversation_history = f"Below is a conversation history,\n{history}\n\n"

In [36]:
result = sa_agent_2.send(conversation_history)

System Analyst # Software Requirements Specification

## 1. Introduction
The purpose of this document is to provide a clear and comprehensive set of software requirements for the development of a custom CRM system for the small cafe owned by the customer. The CRM system aims to enhance customer relationship management, improve operations, and provide valuable insights into customer data and business performance.

## 2. Functional Requirements
### 2.1 Customer Database
The CRM system should include a customer database with the following fields:
- Name
- Contact details (phone number, email address)
- Physical address
- Relevant notes or preferences provided by the customers

### 2.2 Tracking Customer Interactions
The system should allow for the logging and categorization of customer interactions based on the following criteria:
- Type (in-person, phone, email)
- Date of interaction
- Content of the interaction

### 2.3 Loyalty Programs
The CRM system should support the management of a p

In [37]:
pm = {
    "name": "Project Manager",
    "description": "You are a experienced project manager for custom software development.",
}

In [51]:
pm_agent = Agent(pm)

In [52]:
teams = pm_agent.send(
    f"Below is the software requirements specification document,\n{result}\n\n What is the project team you think is suitable for this project?")

Project Manager Based on the software requirements specified, the project team suitable for developing the custom CRM system for the small cafe should include the following roles:

1. Project Manager: Responsible for overall project coordination, communication with the customer, resource allocation, and ensuring timely delivery of the CRM system.

2. Business Analyst: Responsible for gathering and analyzing customer requirements, documenting business processes, and translating them into functional specifications for the development team.

3. Software Architect: Responsible for designing the architecture of the CRM system, ensuring scalability, modularity, and alignment with the customer's business goals.

4. UI/UX Designer: Responsible for creating user-friendly and visually appealing interfaces for the CRM system, focusing on ease of use for both employees and customers.

5. Frontend Developer: Responsible for implementing the UI/UX designs and developing the frontend components of th

In [53]:
 task_break_down = pm_agent.send(f"Hwo will you breakdown the task and assign to each team member?")

Project Manager Breaking down the tasks for developing the custom CRM system for the small cafe and assigning them to the team members could be approached as follows:

1. Project Manager
   - Coordinate with the customer to gather initial requirements and finalize project scope.
   - Allocate resources and create a project timeline.
   - Assign tasks to team members and monitor project progress.
   - Ensure regular communication with the customer and handle any project-related issues.

2. Business Analyst
   - Conduct interviews and workshops with the cafe owner to gather detailed business and functional requirements.
   - Document the business processes and customer requirements.
   - Create functional specifications and use cases for the CRM system.

3. Software Architect
   - Design the overall architecture of the CRM system, considering scalability and modularity.
   - Determine the technology stack and integration points with existing systems.
   - Prepare architectural diagrams a

In [41]:
bd = {
    "name": "Backend Developer",
    "description": "You are a backend developer. You receive document from your project manager. You need to analyse the task then list out the question you need to clarify before kick start your task.",
}

In [45]:
bd_agent = Agent(bd)

In [46]:
task = """### Backend Developer 1
1. Design and develop the customer database with the specified fields.
2. Implement the tracking of customer interactions, including logging and categorization.
3. Develop the loyalty program management functionality."""
required_clarification = bd_agent.send(f"Below is the project break down,\n{project_break_down}\n\n {task}")

Backend Developer Questions to clarify:
1. What are the specified fields for the customer database?
2. What are the requirements for tracking customer interactions? Are there specific data points to capture or categorization criteria to follow?
3. What are the requirements for the loyalty program management functionality? Are there specific rules or features that need to be implemented?

Additionally, it would be helpful to understand the technologies or frameworks to be used and any integration points with other systems.


In [48]:
response_from_sa = sa_agent.send(f"Following are the question from the backend developer, {required_clarification}")

System Analyst To address the backend developer's questions:

1. The specified fields for the customer database include: 
   - Name
   - Contact details (phone number, email address)
   - Physical address
   - Any relevant notes or preferences provided by the customers

2. The requirements for tracking customer interactions involve capturing the following data points and categorization criteria:
   - Type of interaction (in-person, phone, email, etc.)
   - Date and time of interaction
   - Content or details of the interaction
   - Any specific categorization criteria, such as the nature of the interaction (e.g., general inquiry, feedback, complaint, purchase-related, etc.)

3. Regarding loyalty program management functionality, the requirements include:
   - Point-based loyalty program
   - Ability to track and manage customer points
   - Creation and management of different loyalty tiers
   - Implementation of promotions related to the loyalty program
   - Rules for earning and redee

In [49]:
response_from_sa

"To address the backend developer's questions:\n\n1. The specified fields for the customer database include: \n   - Name\n   - Contact details (phone number, email address)\n   - Physical address\n   - Any relevant notes or preferences provided by the customers\n\n2. The requirements for tracking customer interactions involve capturing the following data points and categorization criteria:\n   - Type of interaction (in-person, phone, email, etc.)\n   - Date and time of interaction\n   - Content or details of the interaction\n   - Any specific categorization criteria, such as the nature of the interaction (e.g., general inquiry, feedback, complaint, purchase-related, etc.)\n\n3. Regarding loyalty program management functionality, the requirements include:\n   - Point-based loyalty program\n   - Ability to track and manage customer points\n   - Creation and management of different loyalty tiers\n   - Implementation of promotions related to the loyalty program\n   - Rules for earning and 